tratamento de dados dos dados de diversos paises da netflix

In [54]:
#importação da biblioteca divônica
import pandas as pd

#outras bibliotecas para manipular nossos dados
import xlsxwriter
import openpyxl

#manipular diretorios em massa
import glob

#manipulação do sistema operacional
import os

começando a arrumar
1-confiabilidade e rastreabilidade dos dados
2-extrair
3-transformar
4-carregar

In [55]:
#caminho dos arquivos excel
caminho_pasta='C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw'

#lista todos os arquivos do execel de forma dinamica, juntando todos os arquivos xlsx
arquivos_excel=glob.glob(os.path.join(caminho_pasta,'*.xlsx'))

# Verificação adicional para garantir que encontrou algo
if not arquivos_excel:
    print("Nenhum arquivo .xlsx encontrado no caminho especificado.")
else:
    print(arquivos_excel)

['C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202401_brasil.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202401_france.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202402_brasil.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202402_france.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202403_brasil.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202403_france.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202404_brasil.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202404_france.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202405_brasil.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\raw\\netflix_202405_france.xlsx', 'C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\

In [56]:
#criar vetor de dataframes para guardar os excel
dfs=[]

for arquivo_excel in arquivos_excel:
    try:
        #dataframe temporario para guardar as informações de cada arquivo
        df_temp=pd.read_excel(arquivo_excel)
        
        #pega o nome do arquivo e coloca na nova coluna location
        #poderia ser feito com split, mas pode ser perigoso as futuras nomeações de arquivos, pois se o padrão for diferente, pode contaminar nossa coluna location
        nome_arquivo=os.path.basename(arquivo_excel)
        if 'brasil' in nome_arquivo.lower():
            df_temp['location']='br'
        elif 'france' in nome_arquivo.lower():
            df_temp['location']='fr'    
        elif 'italian' in nome_arquivo.lower():
            df_temp['location']='it' 
            
        #adicionar coluna com o nome do arquivo
        df_temp['file_name']=nome_arquivo
            
        #carregar dados na nova coluna campanha baseado no link que possui a identificação da campanha
        df_temp['campaign']=df_temp['utm_link'].str.extract(r'utm_campaign=(.*)')
        
        #depois de pronto, entra em uma dataframe comum
        dfs.append(df_temp)
        
        #print(df_temp)
       
    except Exception as e:#excesão para erros
        print(f"Erro ao ler arquivo {arquivo_excel}:{e}")



In [57]:
#verifica se ta vazio
if dfs:
    #tranforma tudo em uma tabela concatenada, pra gerar uma tabelona
    result=pd.concat(dfs,ignore_index=True)
    
#print(result)
    

In [58]:
#agora o caminho de saida com o rolê finalizado
arquivo_saida=os.path.join('C:\\Users\\nicol\\Desktop\\etl\\netflix-dataset\\src\\data\\ready','arquivo_limpo.xlsx')
#print(arquivo_saida)

#escreveu o arquivo com o nome e no diretório onde mandamos, em forma de excel
write=pd.ExcelWriter(arquivo_saida,engine='xlsxwriter')

#resultado, que é onde estão nossos dados, são escritos no execel configurado
result.to_excel(write,sheet_name='base',index=True)

#salvando o arquivo excel
write._save()